In [6]:
!apt-get -y install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl
  libfcgi-bin libfcgi-perl libfcgi0ldbl libhtml-parser-perl
  libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2
  libprotobuf-lite23 liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils
  mysql-client-8.0 mysql-client-core-8.0 mysql-server-8.0
  mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl
  mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl
  libfcgi-bin libfcgi-perl libfcgi0ldbl libhtml-parser-perl
  libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2
  lib

In [7]:
!pip install mysql-connector-python

In [8]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [9]:
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'; FLUSH PRIVILEGES;"

In [10]:
!mysql -u root -proot -e "SHOW DATABASES;"

mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


In [11]:
sql_script = """
CREATE DATABASE IF NOT EXISTS empresa;
USE empresa;

CREATE TABLE IF NOT EXISTS funcionarios (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nome VARCHAR(100),
    salario DECIMAL(10,2)
);

DELIMITER //
CREATE FUNCTION calcular_bonus (sal DECIMAL(10,2))
RETURNS DECIMAL(10,2)
DETERMINISTIC
BEGIN
    RETURN sal * 0.10;
END//
DELIMITER ;

DELIMITER //
CREATE PROCEDURE adicionar_funcionario (
    IN p_nome VARCHAR(100),
    IN p_salario DECIMAL(10,2)
)
BEGIN
    INSERT INTO funcionarios (nome, salario) VALUES (p_nome, p_salario);
END//
DELIMITER ;
"""

with open("setup.sql", "w") as f:
    f.write(sql_script)

In [12]:
!mysql -u root -proot < setup.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [13]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="empresa"
)
cursor = conn.cursor()


In [14]:
cursor.callproc('adicionar_funcionario', ['Ana Souza', 3500.00])
conn.commit()

In [15]:
cursor.execute("""
    SELECT nome, salario, calcular_bonus(salario) AS bonus
    FROM funcionarios
""")
for row in cursor.fetchall():
    print(row)

('Ana Souza', Decimal('3500.00'), Decimal('350.00'))


In [16]:
cursor.close()
conn.close()